In [13]:
import sqlite3
netflix = sqlite3.connect('netflix.sqlite')
task = sqlite3.connect("task1.sqlite")
netflix.row_factory = lambda cursor, row: {col[0]: row[i] for i, col in enumerate(cursor.description)}

создаём таблицу фильмов и актёров

In [15]:
task_bd = task.cursor()
with task:
  task_bd.execute("""
  CREATE TABLE filmes (
    film_id INTEGER PRIMARY KEY AUTOINCREMENT,
    type TEXT,
    title TEXT,
    director TEXT,
    country TEXT,
    date_added TEXT,
    release_year INTEGER,
    rating TEXT,
    duration TEXT,
    listed_in TEXT,
    description TEXT
  );
  """)
  task_bd.execute("""
  CREATE TABLE actores (
    actor_id INTEGER PRIMARY KEY AUTOINCREMENT,
    actor_name TEXT UNIQUE
  );
  """)
  task_bd.execute("""
    CREATE TABLE actors_films (
    film_id INTEGER,
    actor_id INTEGER,
    PRIMARY KEY (actor_id, film_id),
    FOREIGN KEY (film_id) REFERENCES films(film_id),
    FOREIGN KEY (actor_id) REFERENCES actors(actor_id)
  );
  """)

Нормализуем 

In [17]:
insert_query = """
INSERT INTO filmes (type, title, listed_in, description)
VALUES (:type, :title, :listed_in, :description);
"""

actors_films_insert_query = """
INSERT OR IGNORE INTO actors_films (film_id, actor_id)
VALUES (?, (SELECT actor_id FROM actores WHERE actor_name = ?));
"""

with netflix, task:
    for row in netflix.execute("SELECT * FROM netflix_titles"):
        task_bd.execute(insert_query, row)
        film_id = task_bd.execute("SELECT last_insert_rowid() FROM filmes;").fetchone()[0]
        
        for actor in row["cast"].split(", "):
            task_bd.execute("INSERT OR IGNORE INTO actors (actor_name) VALUES (?)", (actor,))
            task_bd.execute(actors_films_insert_query, (film_id, actor))

вычисляем пары

In [18]:
with task:
    pair = """
    SELECT a1.actor_name as name1, a2.actor_name COUNT(*) as name2
    FROM actors_films a_f1
    JOIN actors_films a_f2 ON af1.film_id = af_2.film_id AND af_1.actor_id < af_2.actor_id
    JOIN actores a1 ON a_f1.actor_id = a1.actor_id
    JOIN actores a2 ON a_f2.actor_id = a2.actor_id
    GROUP BY a_f1.actor_id, a_f2.actor_id
    ORDER BY name2 DESC
    LIMIT 1;
    """

task_bd.execute(pair)
result = task_bd.fetchone()
print("Наиболее часто работающая пара актеров:", result[0], "и", result[1])

# Закрытие соединения
task.close()